### This notebook is used for the capstone project of IBM data science professional certificate

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [3]:
# scrape information from wikipedia
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [4]:
html_doc = urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").read()

In [5]:
soup = BeautifulSoup(html_doc, 'html.parser')

In [16]:
# define the dataframe column
column_name = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(columns = column_name)


# add the item to the df
for i, tr in enumerate(soup.find_all('tr')):
    if i != 0:
        td = tr.find_all('td')
        if len(td) != 3 :
            break
        if "Not assigned" in td[1].contents[0]:
            continue
        if "Not assigned" in td[2].contents[0]:
            td[2] = td[1]
        if td[1].a != None:
            td[1] = td[1].a
        if td[2].a != None:
            td[2] = td[2].a
        df = df.append({'PostalCode':td[0].contents[0], 'Borough':td[1].contents[0], 'Neighborhood':td[2].contents[0].strip('\n')}, ignore_index = True)
        
            
    

In [17]:
df.tail(30)

,PostalCode,Borough,Neighborhood
180,M9V,Etobicoke,Thistletown
181,M1W,Scarborough,L'Amoreaux West
182,M4W,Downtown Toronto,Rosedale
183,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade
184,M8W,Etobicoke,Alderwood
185,M8W,Etobicoke,Long Branch
186,M9W,Etobicoke,Northwest
187,M1X,Scarborough,Upper Rouge
188,M4X,Downtown Toronto,Cabbagetown
189,M4X,Downtown Toronto,St. James Town


In [18]:
def combine(df):
    return','.join(df.values)
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(combine)
df = df.reset_index()

In [19]:
df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


In [33]:
# get the csv file
geodata = pd.read_csv('Geospatial_Coordinates.csv')


In [41]:
geodata.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)

In [47]:
result = pd.merge(df, geodata,  on = ['PostalCode'])

In [50]:
result

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
